# スターリングの公式を用いて円周率を計算する

スターリングの公式

$$n ! \sim \sqrt{2 \pi n}\left(\frac{n}{e}\right)^n$$

ここで $\sim$ は

$$\lim_{n\to\infty} \frac{n!}{\sqrt{2 \pi n}\left(\frac{n}{e}\right)^n} = 1$$

を満たすことを意味します。両辺 $\sqrt{\pi}$ をかけて

$$\sqrt{\pi} = \lim_{n\to\infty} \frac{n!}{\sqrt{2 n}\left(\frac{n}{e}\right)^n}$$

となります。これで円周率を求めます。

In [37]:
import math
import mpmath

def stirling(n) -> float:
    val = mpmath.mpf(1)
    for i in range(1, n+1):
        val = val * (i * mpmath.e / n)        
    return val / math.sqrt(2 * n) 

In [47]:
DPS = 100
mpmath.mp.dps = DPS
from math import pi

for i in [10, 10**2, 10**3, 10**4, 10**5]:
    mpmath.mp.dps = DPS
    s_pi = stirling(i) ** 2
    mpmath.mp.dps = 8
    print(f"n = {i: <6} \t π ≒ {s_pi} \t 誤差: {s_pi-pi}")

n = 10     	 π ≒ 3.1943736 	 誤差: 0.052780948
n = 100    	 π ≒ 3.146833 	 誤差: 0.005240336
n = 1000   	 π ≒ 3.1421163 	 誤差: 0.00052364239
n = 10000  	 π ≒ 3.141645 	 誤差: 5.2360314e-5
n = 100000 	 π ≒ 3.1415979 	 誤差: 5.2359921e-6


$n$ が $10$ 倍になると誤差が $\frac{1}{10}$ になることがわかります。ただし、この方法だと円周率の上からの評価しか得られないので、もう少し工夫してみます。

$$\lim_{n\to\infty}\frac{n!}{\sqrt{2n}\left(\frac{n}{e}\right)^n} = \sqrt{\pi}$$

を認めて、

$$a_n = \frac{n!}{\sqrt{n}\left(\frac{n}{e}\right)^n}, \quad b_n = \frac{1}{\sqrt{2}}a_n$$

とおくと、
 
$$\begin{align}
\log \frac{b_n}{b_{n+1}} &= \log \left(\frac{n!}{\sqrt{2n}\left(\frac{n}{e}\right)^n} \frac{\sqrt{2(n+1)}\left(\frac{n+1}{e}\right)^{n+1}}{(n+1)!}\right) \\
&= \left(n+\frac{1}{2}\right)\log \left(1+\frac{1}{n}\right) -1
\end{align}$$

となります。テイラー展開 $\log(1+x) = \sum_{k=1}^{\infty} \frac{(-1)^{k-1}}{k}x^k$ $(-1 < x \leq 1)$ を用いると

$$\begin{align}
\log \frac{b_n}{b_{n+1}} &= \left(n+\frac{1}{2}\right)\log \left(1+\frac{1}{n}\right) -1 \\
&= \left(n+\frac{1}{2}\right)\sum_{k=1}^{\infty} \frac{(-1)^{k-1}}{k}\frac{1}{n^k} -1\\
&= \sum_{k=1}^{\infty} \frac{(-1)^{k-1}}{k}\frac{1}{n^{k-1}} +\sum_{k=1}^{\infty} \frac{(-1)^{k-1}}{2k}\frac{1}{n^k} -1\\
&= \sum_{k=2}^{\infty} \frac{(-1)^{k-1}}{k}\frac{1}{n^{k-1}} +\sum_{k=1}^{\infty} \frac{(-1)^{k-1}}{2k}\frac{1}{n^k}\\
&= \sum_{k=2}^{\infty} (-1)^{k}\left(\frac{k -1}{2k(k+1)}\right) \frac{1}{n^{k}} \\
\end{align}$$

となります。$\lim_{n\to\infty} b_n = \sqrt{\pi}$ から

$$\begin{align}
\log \frac{b_n}{\sqrt{\pi}} &= \sum_{m=n}^{\infty} \log \frac{b_m}{b_{m+1}} = \sum_{m=n}^{\infty} \sum_{k=2}^{\infty} (-1)^{k}\left(\frac{k -1}{2k(k+1)}\right) \frac{1}{m^{k}} \\
&= \sum_{k=2}^{\infty} (-1)^{k}\left(\frac{k -1}{2k(k+1)}\right) \sum_{m=n}^{\infty}\frac{1}{m^{k}} 
\end{align}$$

となりますが、

$$\frac{1}{k-1} \frac{1}{n^{k-1}} = \int_{n}^{\infty} \frac{1}{x^k} dx \leq \sum_{m=n}^{\infty}\frac{1}{m^{k}} \leq \int_{n}^{\infty} \frac{1}{x^k} dx + \frac{1}{n^k} = \frac{1}{k-1} \frac{1}{n^{k-1}} + \frac{1}{n^k}$$

が成り立つので

$$\begin{align}
\sum_{k=2}^{\infty} \left(\frac{(-1)^{k}}{2k(k+1)}\right) \frac{1}{n^{k-1}} & \leq \log \frac{b_n}{\sqrt{\pi}} \leq \sum_{k=2}^{\infty} \left(\frac{(-1)^{k}}{2k(k+1)}\right) \frac{1}{n^{k-1}} + \sum_{k=2}^{\infty} (-1)^{k}\left(\frac{k -1}{2k(k+1)}\right) \frac{1}{n^k} \\ 
\frac{1}{12n} -\frac{1}{24n^2} & \leq \log \frac{b_n}{\sqrt{\pi}} \leq \frac{1}{12n} +\frac{1}{12n^2}
\end{align}$$

が成り立ちます。よって

$$\begin{align}
\log b_n -\frac{1}{12n} -\frac{1}{12n^2} &\leq \log \sqrt{\pi} \leq \log b_n -\frac{1}{12n} +\frac{1}{24n^2} \\
b_n e^{-\frac{1}{12n}-\frac{1}{12n^2}} & \leq \sqrt{\pi} \leq b_n e^{-\frac{1}{12n}+\frac{1}{24n^2}} \\
b_n^2 e^{-\frac{1}{6n}-\frac{1}{6n^2}} & \leq \pi \leq  b_n^2 e^{-\frac{1}{6n}+\frac{1}{12n^2}}
\end{align}$$

が成り立ちます。

In [59]:
def modified_stirling_low(n):
    return (stirling(i) ** 2) * mpmath.exp(-1/(6 * n) -1 /(6 * n **2))

def modified_stirling_up(n):
    return stirling(i) ** 2 * mpmath.exp(-1/(6 * n) +1 /(12 * n **2))

In [63]:
DPS = 20

for i in [10, 10**2, 10**3, 10**4, 10**5]:
    mpmath.mp.dps = DPS
    lower = modified_stirling_low(i)
    upper = modified_stirling_up(i)
    print(f"n = {i: <6} \t  {lower} ≦ π ≦ {upper} \t 誤差: {upper - lower}")

n = 10     	  3.1363436519693770038 ≦ π ≦ 3.1441943203458817198 	 誤差: 0.007850668376504715979
n = 100    	  3.1415402766960600994 ≦ π ≦ 3.1416188161847170181 	 誤差: 0.000078539488656918641412
n = 1000   	  3.1415921299736076836 ≦ π ≦ 3.1415929153717383518 	 誤差: 7.8539813066818016493e-7
n = 10000  	  3.1415926483537878037 ≦ π ≦ 3.1415926562077694344 	 誤差: 7.8539816306958580659e-9
n = 100000 	  3.1415926535374334649 ≦ π ≦ 3.1415926536159732813 	 誤差: 7.8539816340263916145e-11


このように $n$ が $10$ 倍で誤差が $2$ 桁減るようになりました。